### Modellarchitektur

Encoder (downsampling): extrahiert die Merkmale (Features) mit Verkleinerung -> Conv2D + MaxPooling2D

Bottleneck: 2 Conv2D Schichten mit 128 Filter, um noch tiefere Merkmale zu erfassen

Decoder (upsampling): erzeugt das Segement, verwendet Concatenate um Features aus dem Encoder hinzuzufügen -> UpSampling2D (skaliert das Bild wieder hoch um das ursprüngliche Format zu rekonstruieren) + Conv2D (verarbeitet die mit Concatenate kombinierten Encoder Informationen)

Output: Grausufen-Bild mit Werten zwischen 0 und 1 == Wahrscheinlickeiten ob Pixel zu Wasser oder nicht-Wasser gehört.

Im Print werden die Schichten des Modells (Layers, Größe der Ausgabe jedes Layers und die Anzahl der Parameter == trainierbare Gewichte an)

Beim Modell selbst handelt es sich um ein unet_model -> is speziell für Image Segmentation in Keras. Neben den oben angeführten Komponenten, enthält es auch noch Skip Connections. https://pyimagesearch.com/2022/02/21/u-net-image-segmentation-in-keras/

Vorteile von U-Net:
- Funktioniert auch gut mit kleinen Datensätzen
- ermöglicht präzise Segmentierungen an Objektgrenzen

Skip Connections: Kopieren die Features vom Encoder in den Decoder

![U-Net Architektur](https://b2633864.smushcdn.com/2633864/wp-content/uploads/2022/02/unet_header-do-not-smush-1.png?size=630x315&lossy=2&strip=1&webp=1)

![U-Net Architektur](https://b2633864.smushcdn.com/2633864/wp-content/uploads/2022/02/1_unet_architecture_paper.png?size=630x350&lossy=2&strip=1&webp=1)

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence

# def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3)):
def unet_model(input_size, dropout_rate=0.2):
  inputs = Input(input_size)

  # Encoder
  c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
  c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
  p1 = MaxPooling2D((2, 2))(c1)
  p1 = Dropout(dropout_rate)(p1)

  c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
  c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
  p2 = MaxPooling2D((2, 2))(c2)
  p2 = Dropout(dropout_rate)(p2)

  c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
  c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
  p3 = MaxPooling2D((2, 2))(c3)
  p3 = Dropout(dropout_rate)(p3)

  # Bottleneck
  c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
  c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)
  c4 = Dropout(dropout_rate * 1.5)(c4)  # Higher dropout in bottleneck

  # Decoder
  u1 = UpSampling2D((2, 2))(c4)
  u1 = Concatenate()([u1, c3])
  c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
  c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)
  c5 = Dropout(dropout_rate)(c5)

  u2 = UpSampling2D((2, 2))(c5)
  u2 = Concatenate()([u2, c2])
  c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
  c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(c6)
  c6 = Dropout(dropout_rate)(c6)

  u3 = UpSampling2D((2, 2))(c6)
  u3 = Concatenate()([u3, c1])
  c7 = Conv2D(16, (3, 3), activation='relu', padding='same')(u3)
  c7 = Conv2D(16, (3, 3), activation='relu', padding='same')(c7)
  c7 = Dropout(dropout_rate)(c7)
  
  outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

  model = Model(inputs, outputs)
  return model